In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, HalvingGridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, median_absolute_error, mean_squared_error
import xgboost as xgb
from scipy.stats import uniform, randint


In [2]:
import h2o
from h2o.automl import H2OAutoML

#### Read in Dataset

In [3]:
df = pd.read_csv('C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataModified/feature_engineered_data.csv', 
                 encoding='utf-8')

C:\Users\sega9\AppData\Local\Temp\ipykernel_7412\1272201554.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataModified/feature_engineered_data.csv',


In [4]:
# List of columns to drop
columns_to_drop = ['title','body','address','latitude','longitude','geometry']
# Drop the specified columns
df = df.drop(columns=columns_to_drop)

#### Encode Data for Modeling

In [5]:
# Columns to be treated as factor
cols = ["bathrooms", "bedrooms", "fee", "has_photo", "cityname", "state", "source"]
df[cols] = df[cols].astype('category')

In [6]:
# One-hot encode categorical variables
data_encoded = pd.get_dummies(df, columns=cols, drop_first=True)

# Display the dimensions, column names, and structure of the DataFrame after encoding
print(data_encoded.shape)
print(data_encoded.columns)
print(data_encoded.info())

(87063, 475)
Index(['id', 'price', 'square_feet', 'zipcode', 'POPULATION', 'POP_SQMI',
       'SQMI', 'perc_sub25k', 'perc_25-50k', 'perc_50-75k',
       ...
       'source_RealRentals', 'source_RentDigs.com', 'source_RentFeeder',
       'source_RentLingo', 'source_Seattle Rentals', 'source_SpreadMyAd',
       'source_Z57', 'source_rentbits', 'source_tenantcloud', 'source_vFlyer'],
      dtype='object', length=475)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87063 entries, 0 to 87062
Columns: 475 entries, id to source_vFlyer
dtypes: float64(11), int64(32), uint8(432)
memory usage: 64.4 MB
None


In [11]:
X = data_encoded.drop(['price'], axis=1)  # 'price' is the dependent variable
y = data_encoded['price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [32]:
# Select ratio
ratio = 0.80
 
total_rows = data_encoded.shape[0]
train_size = int(total_rows*ratio)
 
# Split data into test and train
train = df[0:train_size]
test = df[train_size:]


train.to_csv('C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataModified/train.csv', encoding='utf-8',index= False)
test.to_csv('C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataModified/test.csv', encoding='utf-8',index= False)


### AutoML

In [35]:
train = h2o.import_file('C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataModified/train.csv')
test = h2o.import_file('C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataModified/test.csv')


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.333-b02, mixed mode)
  Starting server from C:\Users\sega9\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\sega9\AppData\Local\Temp\tmp27vavsd8
  JVM stdout: C:\Users\sega9\AppData\Local\Temp\tmp27vavsd8\h2o_sega9_started_from_python.out
  JVM stderr: C:\Users\sega9\AppData\Local\Temp\tmp27vavsd8\h2o_sega9_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,2 months and 12 days
H2O_cluster_name:,H2O_from_python_sega9_1r6c1e
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.420 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [36]:
# Identify predictors and response
x = train.columns
y = "price"
x.remove(y)

In [38]:
# Run AutoML for 20 base models
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
21:00:15.915: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),7/20
# GBM base models (used / total),3/10
# DRF base models (used / total),1/2
# DeepLearning base models (used / total),3/7
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [41]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20240303_210015,417.613,174401,151.965,0.144814,174401
StackedEnsemble_BestOfFamily_1_AutoML_1_20240303_210015,419.05,175603,149.985,0.14406,175603
DRF_1_AutoML_1_20240303_210015,424.171,179921,147.99,0.146818,179921
GBM_grid_1_AutoML_1_20240303_210015_model_1,435.975,190074,158.185,nan,190074
GBM_grid_1_AutoML_1_20240303_210015_model_4,440.112,193699,158.606,nan,193699
GBM_4_AutoML_1_20240303_210015,446.934,199750,178.822,nan,199750
GBM_3_AutoML_1_20240303_210015,449.076,201670,186.981,nan,201670
XRT_1_AutoML_1_20240303_210015,449.285,201857,190.195,0.176326,201857
GBM_5_AutoML_1_20240303_210015,449.778,202300,182.289,nan,202300
GBM_grid_1_AutoML_1_20240303_210015_model_5,451.094,203486,158.312,0.153436,203486


In [42]:
# Get leaderboard with all possible columns
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb

model_id,rmse,mse,mae,rmsle,mean_residual_deviance,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_AllModels_1_AutoML_1_20240303_210015,417.613,174401,151.965,0.144814,174401,1704,0.042786,StackedEnsemble
StackedEnsemble_BestOfFamily_1_AutoML_1_20240303_210015,419.05,175603,149.985,0.14406,175603,1295,0.017969,StackedEnsemble
DRF_1_AutoML_1_20240303_210015,424.171,179921,147.99,0.146818,179921,24566,0.004505,DRF
GBM_grid_1_AutoML_1_20240303_210015_model_1,435.975,190074,158.185,nan,190074,4187,0.005103,GBM
GBM_grid_1_AutoML_1_20240303_210015_model_4,440.112,193699,158.606,nan,193699,6389,0.010373,GBM
GBM_4_AutoML_1_20240303_210015,446.934,199750,178.822,nan,199750,2100,0.003503,GBM
GBM_3_AutoML_1_20240303_210015,449.076,201670,186.981,nan,201670,2233,0.004792,GBM
XRT_1_AutoML_1_20240303_210015,449.285,201857,190.195,0.176326,201857,14981,0.004851,DRF
GBM_5_AutoML_1_20240303_210015,449.778,202300,182.289,nan,202300,2328,0.004874,GBM
GBM_grid_1_AutoML_1_20240303_210015_model_5,451.094,203486,158.312,0.153436,203486,7774,0.005611,GBM


In [46]:
m = h2o.get_model("StackedEnsemble_AllModels_1_AutoML_1_20240303_210015")
# View the parameters for the XGBoost model selected above
m.params.keys()

dict_keys(['model_id', 'training_frame', 'response_column', 'validation_frame', 'blending_frame', 'base_models', 'metalearner_algorithm', 'metalearner_nfolds', 'metalearner_fold_assignment', 'metalearner_fold_column', 'metalearner_params', 'metalearner_transform', 'max_runtime_secs', 'weights_column', 'offset_column', 'custom_metric_func', 'seed', 'score_training_samples', 'keep_levelone_frame', 'export_checkpoints_dir', 'auc_type', 'gainslift_bins'])

In [52]:
# save the model
path="C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/code/h20_model"
model_path = h2o.save_model(model=m, path=path, force=True)
print(model_path)

C:\Users\sega9\Documents\GA Tech\2024.Spring\CSE 6242\Group Project\ApartmentRent\Code\h20_model\StackedEnsemble_AllModels_1_AutoML_1_20240303_210015


In [54]:
# load the model
saved_model = h2o.load_model(model_path)

In [55]:
# upload the model that you just downloded above
# to the H2O cluster
uploaded_model = h2o.upload_model(saved_model)

H2OTypeError: Argument `filename` should be a string, got H2OStackedEnsembleEstimator Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_1_AutoML_1_20240303_210015


Model Summary for Stacked Ensemble: 
key                                        value
-----------------------------------------  ----------------
Stacking strategy                          cross_validation
Number of base models (used / total)       7/20
# GBM base models (used / total)           3/10
# DRF base models (used / total)           1/2
# DeepLearning base models (used / total)  3/7
# GLM base models (used / total)           0/1
Metalearner algorithm                      GLM
Metalearner fold assignment scheme         Random
Metalearner nfolds                         5
Metalearner fold_column
Custom metalearner hyperparameters         None

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 63928.45324124887
RMSE: 252.84076657